<a href="https://colab.research.google.com/github/IzzyBrand/robotBottleFlip/blob/main/traj_opt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import ipywidgets
import pydot
import os
from IPython.display import display, SVG

import importlib
import sys
from urllib.request import urlretrieve

if 'google.colab' in sys.modules and importlib.util.find_spec('manipulation') is None:
    urlretrieve(f"http://manipulation.csail.mit.edu/scripts/setup/setup_manipulation_colab.py",
                "setup_manipulation_colab.py")
    from setup_manipulation_colab import setup_manipulation
    setup_manipulation(manipulation_sha='master', drake_version='20201120', drake_build='nightly')

if 'google.colab' in sys.modules and os.getenv("DISPLAY") is None:
    from pyvirtualdisplay import Display
    display = Display(visible=0, size=(1400, 900))
    display.start()

# Use pyngrok on colab.
server_args = []
if 'google.colab' in sys.modules:
  server_args = ['--ngrok_http_tunnel']

# Start a single meshcat server instance to use for the remainder of this notebook.
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=server_args)

import pydrake.all
from manipulation.scenarios import AddIiwa, AddWsg, AddRgbdSensors, AddShape
from manipulation.utils import FindResource

from pydrake.all import (
    AddMultibodyPlantSceneGraph, ConnectMeshcatVisualizer, DiagramBuilder, 
    FindResourceOrThrow, GenerateHtml, InverseDynamicsController, 
    MultibodyPlant, Parser, Simulator, TrajectorySource, JacobianWrtVariable,
    GenerateHtml)

from pydrake.all import (
    DiagramBuilder,
    RenderEngineVtkParams, MakeRenderEngineVtk, DepthCameraProperties, RgbdSensor, RigidTransform, RollPitchYaw,
    ColorRenderCamera, DepthRenderCamera, CameraInfo, ClippingRange, RenderCameraCore, DepthRange
)
from pydrake.autodiffutils import AutoDiffXd
    
from pydrake.all import RigidTransform, RotationMatrix, SpatialVelocity, PiecewisePolynomial

from pydrake.all import eq, MathematicalProgram, Solve

from pydrake.all import RenderEngineVtkParams, DepthCameraProperties, ModelInstanceIndex, DepthImageToPointCloud, BaseField, LeafSystem, AbstractValue

from google.colab.patches import cv2_imshow

from IPython.display import display, SVG, HTML

In [ ]:
import requests
resp = requests.get("https://raw.githubusercontent.com/IzzyBrand/robotBottleFlip/main/models/bottle.sdf")
if resp.status_code == 200:
    bottle_sdf = resp.text

# Optimize using forward kinematics

In [ ]:
# Define the arguments
X_final = RigidTransform(RotationMatrix(), [0.5, 0, 0.5]) # bottle release pose (RigidTransform)
V_final = np.array([0., -5, 0., 5,  0., 5]) # bottle release velocity (np.ndarray [6])
# q_init = np.array([0, 0.1, 0, -1.2, 0, 1.6, 0])  # iiwa initial pose (np.ndarray [7])
q_init = np.random.randn(7)
X_GB = RigidTransform(RotationMatrix.MakeXRotation(np.pi/2.0).multiply(
        RotationMatrix.MakeZRotation(np.pi/2.0)), [0.0, 0.30, 0.0])    # pose of the bottle in the gripper frame (RigidTransform)
T=100   # number of timestep

max_joint_acceleration = 0.01


# following the example in 
# https://nbviewer.jupyter.org/github/RobotLocomotion/drake/blob/nightly-release/tutorials/mathematical_program_multibody_plant.ipynb

# set up a plant for use in inverse kinematics
plant_f = pydrake.multibody.plant.MultibodyPlant(0.0)
iiwa = AddIiwa(plant_f)
plant_f.Finalize()
context_f = plant_f.CreateDefaultContext()
plant_ad = plant_f.ToAutoDiffXd()
context_ad = plant_ad.CreateDefaultContext()
W = plant_f.world_frame()
L7 = plant_f.GetBodyByName('iiwa_link_7') # get the last link for later use

# transform from link 7 to the WSG
X_L7G = RigidTransform(RollPitchYaw(np.pi / 2.0, 0, np.pi / 2.0), [0, 0, 0.114])
# and from link 7 to the bottle. Note that this needs to be an Expression
# to use in the symbolic computations
X_L7B = X_L7G.multiply(X_GB)

# set up the optimization problem
prog = MathematicalProgram()

# with optimization variables
q = prog.NewContinuousVariables(T, 7, 'q')     # joint position
qd = prog.NewContinuousVariables(T, 7, 'qd')   # joint velocity
h = prog.NewContinuousVariables(1, 'h')        # timestep

# put the plant at the final state
# plant.SetPositions(context, q[-1])
# plant.SetVelocities(context, qd[-1])

def vectorize(X, V):
    return np.hstack([np.ravel(X.rotation().matrix()), X.translation(), V])


def resolve_frame(plant, F):
    """Gets a frame from a plant whose scalar type may be different."""
    return plant.GetFrameByName(F.name(), F.model_instance())

vectorized_final_state = vectorize(X_final, V_final)

def forward_kinematics(vars):

    # Choose plant and context based on dtype.
    if vars.dtype == float:
        plant_local = plant_f
        context = context_f
        X_L7B_local = X_L7B
    else:
        # Assume AutoDiff.
        plant_local = plant_ad
        context = context_ad
        X_L7B_local = X_L7B.cast[AutoDiffXd]()
    
    q = vars[:7]
    qd = vars[7:]

    plant_local.SetPositions(context, q)
    plant_local.SetVelocities(context, qd)
    

    X_WL7 = plant_local.CalcRelativeTransform(
        context, resolve_frame(plant_local, W), resolve_frame(plant_local, L7))
    X_WB = X_WL7.multiply(X_L7B_local)
    J_B = plant_local.CalcJacobianSpatialVelocity(context,
                                            JacobianWrtVariable.kV,
                                            resolve_frame(plant_local, L7),
                                            X_L7B_local.translation(),
                                            resolve_frame(plant_local, W), resolve_frame(plant_local, W))
    
    V_WB = J_B.dot(qd)

    return np.sum((vectorize(X_WB, V_WB) - vectorized_final_state)**2)

def get_acceleration(vars):
    qd1 = vars[:7]
    qd2 = vars[7:-1]
    h = vars[-1]

    return (qd1 - qd2)/h

prog.AddCost(forward_kinematics, vars=np.hstack([q[-1], qd[-1]]))
# prog.AddConstraint(forward_kinematics,
#                    lb=vectorized_final_state - 1e-2,
#                    ub=vectorized_final_state + 1e-2,
#                    vars=np.hstack([q[-1], qd[-1]]))

# # compute the resulting bottle pose and velocity
# X_WL7 = plant.EvalBodyPoseInWorld(context , L7) # pose of link 7 (symbolic)
# X_WB = X_WL7.multiply(X_L7B.cast[AutoDiffXd]()) # pose of the bottle at the final timestep

# # compute the spatial velocity jacobian of the bottle
# J_B = plant.CalcJacobianSpatialVelocity(context,
#                                         JacobianWrtVariable.kQDot, # this often is .kv
#                                         L7.body_frame(),
#                                         X_L7B.translation(),
#                                         plant.world_frame(),
#                                         plant.world_frame())

# and finally, the spatial velocity of the bottle at the last timestep
# V_WB = J_B.dot(qd[-1]) # [roll, pitch, yaw, x, y, z]

# add trajectory constraints
# NOTE(izzy): for some reason I can't make these AddLinearConstraint...
for t in range(T-1):
    prog.AddConstraint(eq(q[t+1], q[t] + h*qd[t+1]))    # position
    # prog.AddConstraint(np.ones(7)*max_joint_acceleration >= qd[t] - qd[t+1])
    prog.AddConstraint(get_acceleration,
                                  lb = -np.ones(7)*max_joint_acceleration,
                                  ub = np.ones(7)*max_joint_acceleration,
                                  vars=np.hstack([qd[t], qd[t+1], h]))
    # prog.AddConstraint(np.ones(7)*max_joint_acceleration >= np.abs((qd[t+1] - qd[t])/h)) # bound the joint accelerations
# prog.AddConstraint(eq(q[1:], q[:-1] + h*qd[:-1]))    # position
# prog.AddConstraint(eq(qd[1:], qd[:-1] + h*qdd[:-1])) # velocity
prog.AddConstraint(eq(q[0], q_init)) # initial joint position

# add timestep constraint
prog.AddBoundingBoxConstraint(0.01, 0.02, h)

# add final position constraint
# print('adding pose constraints')
# for i in range(3):
#     prog.AddConstraint(eq(X_WB.rotation().matrix()[i], X_final.rotation().matrix()[i]))
#     print(i)


# print('adding velocity constraint')
# # add final velocity constraint
# prog.AddConstraint(eq(V_WB, V_final))
# print('done')

# add bottle acceleration constraints
# TODO

# add joint acceleration costs
# TODO

result = Solve(prog)
if result.is_success():
    q_opt = result.GetSolution(q)
    qd_opt = result.GetSolution(qd)
    x_opt = np.hstack([q_opt, qd_opt])
    h_opt = result.GetSolution(h)
    print('SUCCESS')
else:
    print('Optimization Failed')

#Optimize using inverse kinematics

In [ ]:
# NOTE(izzy): the goal here was to use pydrake's IK, but that doesn't allow us
# to add velocity constraint

# X_final = RigidTransform(RotationMatrix(), [0.5, 0.5, 0.5])
# q_init = np.zeros(7) # TODO: get initial iiwa pose
# X_GB = RigidTransform()
# T = 50

# # create a plant to be used in control
# plant = pydrake.multibody.plant.MultibodyPlant(0.0)
# iiwa = AddIiwa(plant)
# wsg = AddWsg(plant, iiwa) # NOTE : we might now want to include this
# plant.Finalize()

# # create an ik solver for that plant
# ik = pydrake.multibody.inverse_kinematics.InverseKinematics(plant)

# # constrain the final position
# ik.AddPositionConstraint(
#     frameA=plant.world_frame(), # world frame
#     frameB=plant.GetFrameByName('body'), # gripper frame
#     p_BQ=X_GB.translation(), # position of the bottle in the gripper frame
#     p_AQ_lower=X_final.translation(), # should be at the final position
#     p_AQ_upper=X_final.translation())

# # get the ik's pose
# q_final = ik.q()[:7] # we don't care about the pose of the fingers

# # now convert this to a regular mathematical program
# prog = ik.prog()
# q = prog.NewContinuousVariables(T, 7, 'q')     # joint position
# qd = prog.NewContinuousVariables(T, 7, 'qd')   # joint velocity
# h = prog.NewContinuousVariables(1, 'h')        # timestep

# # the trajectory starts and ends in the right place
# prog.AddConstraint(eq(q[0], q_init)) 
# prog.AddConstraint(eq(q[-1], q_final))

# # add dynamics constraints
# for t in range(T-1):
#     prog.AddConstraint(eq(q[t+1], q[t] + h*qd[t+1]))    # position

# # add timestep constraint
# prog.AddBoundingBoxConstraint(0.001, 0.02, h)

# result = Solve(prog)
# if result.is_success():
#     q_opt = result.GetSolution(q)
#     qd_opt = result.GetSolution(qd)
#     x_opt = np.hstack([q_opt, qd_opt])
#     h_opt = result.GetSolution(h)
# else:
#     print('Optimization Failed')

In [ ]:
def MakeManipulationStation(time_step=0.001):
    builder = pydrake.systems.framework.DiagramBuilder()

    # Add the iiwa, WSG, and cameras to the scene.
    plant, scene_graph = pydrake.multibody.plant.AddMultibodyPlantSceneGraph(
        builder, time_step=time_step)
    iiwa = AddIiwa(plant)
    wsg = AddWsg(plant, iiwa)
    plant.Finalize()

    num_iiwa_positions = plant.num_positions(iiwa)


    # Export the iiwa "state" outputs.
    demux = builder.AddSystem(pydrake.systems.primitives.Demultiplexer(
        2 * num_iiwa_positions, num_iiwa_positions))
    builder.Connect(plant.get_state_output_port(iiwa), demux.get_input_port())
    builder.ExportOutput(demux.get_output_port(0), "iiwa_position_measured")
    builder.ExportOutput(demux.get_output_port(1), "iiwa_velocity_estimated")
    builder.ExportOutput(plant.get_state_output_port(iiwa), "iiwa_state_estimated")

    # Make the plant for the iiwa controller to use.
    controller_plant = pydrake.multibody.plant.MultibodyPlant(time_step=time_step)
    controller_iiwa = AddIiwa(controller_plant)
    AddWsg(controller_plant, controller_iiwa, welded=True)
    controller_plant.Finalize()

    # Add the iiwa controller
    iiwa_controller = builder.AddSystem(
        pydrake.systems.controllers.InverseDynamicsController(
            controller_plant,
            kp=[100]*num_iiwa_positions,
            ki=[1]*num_iiwa_positions,
            kd=[20]*num_iiwa_positions,
            has_reference_acceleration=False))
    iiwa_controller.set_name("iiwa_controller")
    builder.Connect(
        plant.get_state_output_port(iiwa), iiwa_controller.get_input_port_estimated_state())
    
    # NOTE(izzy): expose the iiwa controller and connect it directly to the iiwa input
    builder.ExportInput(iiwa_controller.get_input_port_desired_state(), 'iiwa_desired_state')
    builder.Connect(iiwa_controller.get_output_port(), plant.get_actuation_input_port(iiwa))

    print(plant.GetInputPort('iiwa7_actuation'))
    # print(iiwa.GetInputPort('iiwa7_actuation'))
    print(plant.get_actuation_input_port(iiwa))


    # Wsg controller.
    wsg_controller = builder.AddSystem(pydrake.manipulation.schunk_wsg.SchunkWsgPositionController())
    wsg_controller.set_name("wsg_controller")
    builder.Connect(wsg_controller.get_generalized_force_output_port(),             
                    plant.get_actuation_input_port(wsg))
    builder.Connect(plant.get_state_output_port(wsg), wsg_controller.get_state_input_port())
    builder.ExportInput(wsg_controller.get_desired_position_input_port(), "wsg_position")
    builder.ExportInput(wsg_controller.get_force_limit_input_port(), "wsg_force_limit")
    wsg_mbp_state_to_wsg_state = builder.AddSystem(
        pydrake.manipulation.schunk_wsg.MakeMultibodyStateToWsgStateSystem())
    builder.Connect(plant.get_state_output_port(wsg), wsg_mbp_state_to_wsg_state.get_input_port())
    builder.ExportOutput(wsg_mbp_state_to_wsg_state.get_output_port(), "wsg_state_measured")
    builder.ExportOutput(wsg_controller.get_grip_force_output_port(), "wsg_force_measured")


    # Export "cheat" ports.
    builder.ExportOutput(scene_graph.get_query_output_port(), "geometry_query")
    builder.ExportOutput(plant.get_contact_results_output_port(), "contact_results")
    builder.ExportOutput(plant.get_state_output_port(), "plant_continuous_state")

    return builder.Build()

In [ ]:
builder = pydrake.systems.framework.DiagramBuilder()
station = builder.AddSystem(MakeManipulationStation())
station.set_name('station')
meshcat = ConnectMeshcatVisualizer(builder,
                                   output_port=station.GetOutputPort("geometry_query"),
                                   zmq_url=zmq_url)

times = np.linspace(0, h_opt.item()*T, T)
traj_iiwa_command = PiecewisePolynomial.CubicShapePreserving(times, x_opt.T)
iiwa_source = builder.AddSystem(TrajectorySource(traj_iiwa_command))
iiwa_source.set_name("iiwa_source")
builder.Connect(iiwa_source.get_output_port(), station.GetInputPort("iiwa_desired_state"))

diagram = builder.Build()
context = diagram.CreateDefaultContext()
simulator = Simulator(diagram)

station_mutable_context = station.GetMyMutableContextFromRoot(simulator.get_mutable_context())
station.GetSubsystemByName('plant').SetPositions(station_mutable_context, np.hstack([q_init, [-0.01, 0.01]]))

In [ ]:
# HTML('<script src="https://unpkg.com/gojs/release/go.js"></script>' + GenerateHtml(station))

In [ ]:
meshcat.start_recording()
simulator.AdvanceTo(5)
meshcat.publish_recording()

In [ ]:
station.GetSubsystemByName('plant')